# Opening


### Data Wrangling 


##### Import necessary libraries 

In [1]:
import pandas as pd 
import numpy as np 
from datetime import datetime,timedelta

Import the tables

In [2]:
customer_cases = pd.read_csv(r'G:\My Drive\Lily Career\Work\Portfolio\Power BI - CLV\Dataset\customer_cases.csv')
customer_info = pd.read_csv(r'G:\My Drive\Lily Career\Work\Portfolio\Power BI - CLV\Dataset\customer_info.csv')
customer_product = pd.read_csv(r'G:\My Drive\Lily Career\Work\Portfolio\Power BI - CLV\Dataset\customer_product.csv')
product_info = pd.read_csv(r'G:\My Drive\Lily Career\Work\Portfolio\Power BI - CLV\Dataset\product_info.csv')

In [3]:
product_info.reset_index(drop=True,inplace=True)
customer_cases.reset_index(drop=True,inplace=True)
customer_info.reset_index(drop=True,inplace=True)
customer_product.reset_index(drop=True,inplace=True)

customer_product.sort_values(by='signup_date_time',ascending=True,inplace=True)
customer_product.head()

,Unnamed: 0,customer_id,product,signup_date_time,cancel_date_time
129527,129528,C131975,prd_2,2017-01-01 07:55:42,NaN
129528,129529,C131976,prd_1,2017-01-01 09:16:32,NaN
129529,129530,C131977,prd_1,2017-01-01 09:37:09,NaN
129530,129531,C131978,prd_1,2017-01-01 10:14:28,2020-10-21 15:39:59
0,1,C2448,prd_1,2017-01-01 10:35:09,NaN


Data Cleaning & Transformation 

#### "customer_case" table

In [4]:
customer_cases['date_time'] = pd.to_datetime(customer_cases['date_time'])
customer_cases.drop(columns=['Unnamed: 0'],inplace=True)
customer_cases.to_csv('customer_cases_date.csv',index=False)

##### "customer_product" table

In [7]:
# Remove "Unnames" column and duplicate rows
customer_product.drop(columns=['Unnamed: 0'],inplace=True)
customer_product.drop_duplicates(inplace=True)
customer_product.reset_index(drop=True,inplace=True)

customer_product.info()

,customer_id,product,signup_date_time,cancel_date_time
0,C131975,prd_2,2017-01-01 07:55:42,NaN
1,C131976,prd_1,2017-01-01 09:16:32,NaN
2,C131977,prd_1,2017-01-01 09:37:09,NaN
3,C131978,prd_1,2017-01-01 10:14:28,2020-10-21 15:39:59
4,C2448,prd_1,2017-01-01 10:35:09,NaN


In [5]:
customer_product['signup_date_time'] = pd.to_datetime(customer_product['signup_date_time'])
customer_product['cancel_date_time'] = pd.to_datetime(customer_product['cancel_date_time'])
customer_product.info()

<class 'pandas.core.frame.DataFrame'>
Index: 508932 entries, 129527 to 508931
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Unnamed: 0        508932 non-null  int64         
 1   customer_id       508932 non-null  object        
 2   product           508932 non-null  object        
 3   signup_date_time  508932 non-null  datetime64[ns]
 4   cancel_date_time  112485 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 23.3+ MB


In [7]:
customer_product.to_csv('customer_product_clean.csv',index=False)

##### Create and populate new transaction columns 

In [14]:
# Populate new "customer_product" table 
new_cus_prod = pd.merge(customer_product,product_info,left_on='product',right_on='product_id')
new_cus_prod = new_cus_prod.drop(columns=['product_id','name'])

In [20]:
def transactions_table(df):
    transactions = []
    end_period = pd.to_datetime(df[['signup_date_time','cancel_date_time']].max().max())
    
    for index,row in df.iterrows():
        customer_id = row['customer_id']
        product = row['product']
        signup_date_time = row['signup_date_time']
        cancel_date_time = row['cancel_date_time']
        amount = row['price']
        billing_cycle  = row['billing_cycle']
        if pd.isna(cancel_date_time):
            end_date = end_period
        else:
            end_date = cancel_date_time
        
        transaction_date = signup_date_time
        while transaction_date <= end_date and transaction_date <= end_period:
            transactions.append([customer_id,transaction_date,signup_date_time,cancel_date_time,product,amount])
            transaction_date += timedelta(days=30 if product == 'prd_2' else 365)
    return pd.DataFrame(transactions, columns=['customer_id','transaction_date','signup_date_time','cancel_date_time','product','amount'])

transactions_df = transactions_table(new_cus_prod)

In [21]:
transactions_df.head(10)

,customer_id,transaction_date,signup_date_time,cancel_date_time,product,amount
0,C131975,2017-01-01 07:55:42,2017-01-01 07:55:42,NaT,prd_2,125
1,C131975,2017-01-31 07:55:42,2017-01-01 07:55:42,NaT,prd_2,125
2,C131975,2017-03-02 07:55:42,2017-01-01 07:55:42,NaT,prd_2,125
3,C131975,2017-04-01 07:55:42,2017-01-01 07:55:42,NaT,prd_2,125
4,C131975,2017-05-01 07:55:42,2017-01-01 07:55:42,NaT,prd_2,125
5,C131975,2017-05-31 07:55:42,2017-01-01 07:55:42,NaT,prd_2,125
6,C131975,2017-06-30 07:55:42,2017-01-01 07:55:42,NaT,prd_2,125
7,C131975,2017-07-30 07:55:42,2017-01-01 07:55:42,NaT,prd_2,125
8,C131975,2017-08-29 07:55:42,2017-01-01 07:55:42,NaT,prd_2,125
9,C131975,2017-09-28 07:55:42,2017-01-01 07:55:42,NaT,prd_2,125


In [30]:
transactions_df.to_csv('transactions_table.csv',index=False)

In [40]:
start = pd.to_datetime('2017-01-01')
end = pd.to_datetime('2017-01-31')
filter = customer_product[(customer_product['signup_date_time'].between(start,end))]
filter2 = transactions_df[(transactions_df['transaction_date'].between(start,end))]
print(filter2['customer_id'].nunique())
filter.shape[0]

3577


3577